# Project 3


## Importing Modules and staring local db engine

In [ ]:
#Importing modules

import boto3

from data_cleaning import DataCleaning
from data_extraction import DataExtractor
from database_utils import DatabaseConnector
import importlib
import json
import numpy as np
import pandas as pd
import re
import requests
from sqlalchemy import create_engine
from sqlalchemy import inspect
import tabula
import yaml


#starting local database

#read creds
uploader = DatabaseConnector()
ml_yaml = 'ml_dbs.yaml'
uploader.read_db_creds(ml_yaml)

# initialising and returning an sqlalchemy database engine.

uploading = uploader.init_db_engine_postgresql()



## RDS ETL: User


In [ ]:
### 1. Getting data from the RDS database

## read credentials
connector_rds = DatabaseConnector()
ai_core_yaml = 'db_creds.yaml'
connector_rds.read_db_creds(ai_core_yaml)

## initialising and returning an sqlalchemy database engine.
connection_rds = connector_rds.init_db_engine()

## Reading the data from the RDS database
connector_tables_rds = connector_rds.list_db_tables()


### 2. Extracting the 'legacy_users' table to a pandas DataFrame.
user_table_rds = 'legacy_users'
extractor_rds = DataExtractor(connection_rds)
user_df_rds = extractor_rds.read_rds_table(user_table_rds)
print(user_df_rds.head(5))
print(user_df_rds.info())



In [ ]:
### 3.Cleaning data
clean_data_rds = DataCleaning()
clean_df_rds = clean_data_rds.clean_user_data(user_df_rds)
clean_dob = clean_data_rds.date_formatting(clean_df_rds, 'date_of_birth')

clean_join_date = clean_data_rds.date_formatting(clean_dob,'join_date')
clean_email = clean_data_rds.email_clean(clean_join_date, 'email_address')
clean_first_name = clean_data_rds.gibberish_clean(clean_email, 'first_name')
clean_last_name = clean_data_rds.gibberish_clean(clean_first_name, 'last_name')
clean_dim_users = clean_data_rds.null_replace(clean_last_name)


In [ ]:
### 4. Loading the data
#load clean rds dataframe into local database with a new table
uploader.upload_to_db(uploading, clean_df_rds, 'dim_users')

## PDF ETL: Card Details

In [ ]:
### 1.Getting the data from PDF & load in df
connector_pdf = DatabaseConnector()
extractor_pdf = DataExtractor(connector_pdf)

link = "https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf"
extraction_pdf = extractor_pdf.retrieve_pdf_data(link)
print(extraction_pdf.head(5))
print(extraction_pdf.info())

### 2.Clean the pdf df

clean_pdf = DataCleaning()

clean_df_pdf =clean_pdf.clean_card_data(extraction_pdf)

### 3. load into db

uploader.upload_to_db(uploading, clean_df_pdf, 'dim_card_details')

## API ETL:  Store







In [2]:
### 1. Getting Data from API


headers = {
    'x-api-key': 'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'
}

number_of_stores_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'
store_details_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/{store_number}'

connector_api = DatabaseConnector()
extractor_api = DataExtractor(connector_api)


number_of_stores = extractor_api.list_number_of_stores(number_of_stores_endpoint, headers)
print(number_of_stores)


### 2. Extracting the table to a pandas DataFrame

stores_df = extractor_api.retrieve_stores_data(store_details_endpoint, headers, number_of_stores)


Failed to retrieve data for store number 451. Status code: 500
Failed to retrieve data for store number 452. Status code: 500
   index                                            address longitude   lat  \
0      1  Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...  51.62907  None   
1      2        Heckerstraße 4/5\n50491 Säckingen, Landshut  48.52961  None   
2      3  5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury     51.26  None   
3      4  Studio 6\nStephen landing\nSouth Simon\nB77 2W...   53.0233  None   
4      5  Flat 92u\nChristian harbors\nPort Charlotte\nN...  53.38333  None   
5      6  7 Gillian rue\nWest Robertside\nPH4 8NY, Ruthe...  55.82885  None   
6      7  Lilija-Heß-Allee 660\n34566 Regensburg, Stuttgart  48.78232  None   
7      8     510 Jill Mill\nSouth Laura, FL 38723, Kaukauna  44.27804  None   
8      9   3 Lee valleys\nWest Janetview\nDY4M 2RL, Hartley  51.38673  None   
9     10  8 Gareth skyway\nSimmonsview\nSW4 7PL, Rutherglen  55.82885  None   

    

In [ ]:
print(stores_df.head(10))



In [3]:
### 3.Cleaning data

clean_stores_api = DataCleaning()

replace_nulls_api = clean_stores_api.null_replace(stores_df)






In [12]:


drop_df = replace_nulls_api.dropna(axis=0, thresh=2)
drop_df_columns = drop_df.dropna(axis = 1, thresh = 100)

print(drop_df.info())

drop_df_columns.info()

### 4. Loading the data
uploader.upload_to_db(uploading,drop_df_columns, 'dim_store_details')

<class 'pandas.core.frame.DataFrame'>
Index: 447 entries, 0 to 449
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          447 non-null    int64 
 1   address        447 non-null    object
 2   longitude      447 non-null    object
 3   lat            7 non-null      object
 4   locality       447 non-null    object
 5   store_code     447 non-null    object
 6   staff_numbers  447 non-null    object
 7   opening_date   447 non-null    object
 8   store_type     447 non-null    object
 9   latitude       447 non-null    object
 10  country_code   447 non-null    object
 11  continent      447 non-null    object
dtypes: int64(1), object(11)
memory usage: 45.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 447 entries, 0 to 449
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          447 non-null    int64 
 1   address

## S3 ETL: Products


In [ ]:
### 1. Getting data

connector_s3 = DatabaseConnector()
extractor_s3 = DataExtractor(connector_s3)


### 2. Putting into pd df

bucket = 'data-handling-public'
object_key = "products.csv"
pathway = '/Users/student/AICORE/AWS/Project_3/products.csv'

extraction_s3 = extractor_s3.download_csv_from_s3(bucket, object_key, pathway)
print(extraction_s3.head(20))
print(extraction_s3.info())

### 3. Cleaning data

clean_s3 = DataCleaning()

clean_df_s3 = clean_s3.called_clean_store_data(extraction_s3)


clean_df_s3 = clean_s3.called_clean_store_data(extraction_s3)


converted_df_s3 = clean_s3.convert_product_weights(clean_df_s3)
print(converted_df_s3.head())


clean_df_s3 = clean_s3.clean_products_data(converted_df_s3)


### 4. Loading into local db
uploader.upload_to_db(uploading, clean_df_s3, 'dim_products')


## AWS DB ETL: Orders

In [ ]:
connector_tables_rds = connector_rds.list_db_tables()

#'orders_table'


In [ ]:
orders_table_rds = 'orders_table'

extractor_rds = DataExtractor(connection_rds)

orders_df_rds = extractor_rds.read_rds_table(orders_table_rds)
print(orders_df_rds.head(5))
print(orders_df_rds.info())

#cleaning the orders table

clean_ordersdf_rds = clean_data_rds.clean_user_data(orders_df_rds)

## dropping columns

ordersdf_rds = clean_ordersdf_rds.drop(['first_name','last_name','1' ], axis = 1)
print(ordersdf_rds.info())

#uploading the updated orders table into the local db
uploader.upload_to_db(uploading, ordersdf_rds, 'orders_table')


## JSON S3 ETL: Date Details

In [ ]:
#Preparing the extration

connector_json_s3 = DatabaseConnector()
extractor_json_s3 = DataExtractor(connector_json_s3)
bucket = 'data-handling-public'
object_key_json = "date_details.json"

#Get the object

extraction_json_s3 = extractor_json_s3.extract_json_from_s3(bucket, object_key_json)
print(extraction_json_s3.head(20))
print(extraction_json_s3.info())

#Clean the data
clean_json_s3 = DataCleaning()

clean_df_json_s3 = clean_json_s3.called_clean_store_data(extraction_json_s3)
clean_df_date = clean_json_s3.null_replace(extraction_json_s3)

#upload to local db
uploader.upload_to_db(uploading, clean_df_json_s3, 'dim_date_times')

In [17]:


#stores_df.store_type.unique()



array(['Local', 'Super Store', 'Mall Kiosk', 'Outlet', 'QP74AHEQT0',
       'O0QJIRC943', 'NULL', '50IB01SFAZ', '0RSNUU3DF5', 'B4KVQB3P5Y',
       'X0FE7E2EOG', 'NN04B3F6UQ'], dtype=object)